# Read results processed by MintPy

In [ ]:
import os
import h5py
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def read_h5(fname, label):
    with h5py.File(fname, 'r') as f:
        atr = dict(f.attrs)
        data = np.asarray(f[(label)])
    return data, atr

## read velocity

**read velocity and mask files, return 4 colums data (num lon lat velocity), you also can save data**

In [ ]:
def read_vel(vel_file, mask_file, out_vel_file=None):
    # read velocity
    vel, atr = read_h5(vel_file, 'velocity')
    # get longitude and latitude of the upperleft corner
    lon = float(atr['X_FIRST'])
    lat = float(atr['Y_FIRST'])
    # get interval of lonitude and latitude
    lon_step = float(atr['X_STEP'])
    lat_step = float(atr['Y_STEP'])
    # generate latitude and longitude coordinates
    lon_tmp = np.linspace(lon, lon + lon_step * vel.shape[1], vel.shape[1])
    lat_tmp = np.linspace(lat, lat + lat_step * vel.shape[0], vel.shape[0])
    lons, lats = np.meshgrid(lon_tmp, lat_tmp)
    # read mask
    mask, _ = read_h5(mask_file, 'mask')
    # mask and reshape
    lons = lons[mask].reshape((-1, 1))
    lats = lats[mask].reshape((-1, 1))
    vel = vel[mask].reshape((-1, 1))
    # generate point ID
    num = np.arange(vel.shape[0]).reshape((-1, 1))
    # m/yr to mm/yr
    vel *= 1000
    # print some information
    print('max velocity : ', np.max(vel))
    print('min velocity : ', np.min(vel))
    print('number of points : ', vel.shape[0])
    # save data
    out_data = np.hstack((num, lons, lats, vel))
    if out_vel_file:
        print('writing data to {}'.format(out_vel_file))
        np.savetxt(out_vel_file, out_data, fmt='%4f')
        print('done.')
    return out_data

In [ ]:
# velocity = read_vel('geo_velocity.h5', 'geo_maskTempCoh.h5', 'vels.txt')
velocity = read_vel('geo_velocity.h5', 'geo_maskTempCoh.h5')

## downsample

**random downsample data in [min, max], return 4 colums data (num lon lat velocity), you also can save data**

In [ ]:
def random_downsample(data, min_vel, max_vel, rate, out_file=None):
    if min_vel > max_vel:
        tmp = min_vel
        min_vel = max_vel
        max_vel = tmp
    #  vel <= min_vel
    less_min = data[:, 3] <= min_vel
    data_less_min = data[less_min, :]
    print(f"number of velocity < {min_vel} : {data_less_min.shape[0]}")
    # vel >= max_vel
    more_max = data[:, 3] >= max_vel
    data_more_max = data[more_max, :]
    print(f"number of velocity > {max_vel} : {data_more_max.shape[0]}")
    # vel > min_vel and vel < max_vel
    min_max = (less_min == more_max)
    data_min_max = data[min_max, :]
    print(
        f"number of velocity in [{min_vel}, {max_vel}] : {data_min_max.shape[0]}")
    # downsample vel > min_vel and vel < max_vel
    index = random.sample(range(data_min_max.shape[0]), int(
        data_min_max.shape[0] * rate))
    sampled_data = data_min_max[index, :]
    # save data
    out_data = np.vstack((data_less_min, data_more_max, sampled_data))
    print('max velocity : ', np.max(out_data[:, 3]))
    print('min velocity : ', np.min(out_data[:, 3]))
    print('number of points : ', out_data.shape[0])
    if out_file:
        print('writing data to {}'.format(out_file))
        np.savetxt(out_file, out_data, fmt='%4f')
        print('done.')
    return out_data

In [ ]:
# vel_ds = random_downsample(velocity, 10, -10, 0.3, "vels_ds.txt")
vel_ds = random_downsample(velocity, 10, -10, 0.3)

## read time-series and velocity

**read ts_file, vel_file, mask_file, return timeseries data(num lon lat vel disp1 disp2 ...) and date, you also can save data**

In [ ]:
def read_ts(ts_file, vel_file, mask_file, out_vel_file=None, out_ts_file=None):
    mask, _ = read_h5(mask_file, 'mask')
    mask = np.asarray(mask)

    vel, _ = read_h5(vel_file, 'velocity')
    vel = np.asarray(vel) * 1000

    date, _ = read_h5(ts_file, 'date')
    date = date.astype(np.int64)
    ts, atr = read_h5(ts_file, 'timeseries')
    ts = np.asarray(ts)
    ts = ts.reshape((date.shape[0], -1, 1)) * 1000

    lon = float(atr['X_FIRST'])
    lon_step = float(atr['X_STEP'])

    lat = float(atr['Y_FIRST'])
    lat_step = float(atr['Y_STEP'])

    lon_tmp = np.linspace(lon, lon + lon_step * vel.shape[1], vel.shape[1])
    lat_tmp = np.linspace(lat, lat + lat_step * vel.shape[0], vel.shape[0])

    lons, lats = np.meshgrid(lon_tmp, lat_tmp)

    lons = lons.reshape((-1, 1))
    lats = lats.reshape((-1, 1))
    vels = vel.reshape((-1, 1))
    mask = mask.reshape((-1, 1))

    lons = lons[mask].reshape((-1, 1))
    lats = lats[mask].reshape((-1, 1))
    vels = vels[mask].reshape((-1, 1))
    num = np.arange(lons.shape[0]).reshape((-1, 1))

    out_vel = np.hstack((num, lons, lats, vels))
    if out_vel_file:
        # save lon, lat, velocity
        print('writing data to {}'.format(out_vel_file))
        np.savetxt(out_vel_file, out_vel, fmt='%4f')
        print('done.')

    out_ts = out_vel
    for i in range(ts.shape[0]):
        data = ts[i]
        out_ts = np.hstack((out_ts, data[mask].reshape((-1, 1))))

    tmp = out_ts[:, 4:]
    tmp = tmp - tmp[:, 0].reshape((-1, 1))
    tmp = np.hstack((out_vel, tmp))

    out_ts = tmp
    if out_ts_file:
        # save lon, lat, vel, disp
        print('writing data to {}'.format(out_ts_file))
        np.savetxt(out_ts_file, out_ts, fmt='%4f')
        print('done.')

    return out_ts, date

In [ ]:
os.chdir('/media/ly/file/读取MintPy结果')
ts = 'geo_timeseries_tropHgt_ramp_demErr.h5'
vel = 'geo_velocity.h5'
mask = 'geo_maskTempCoh.h5'

# ts_data, date =read_ts(ts, vel, mask, 'vel.txt', 'ts.txt')
ts_data, date = read_ts(ts, vel, mask)

In [ ]:
# sampled_ts = random_downsample(ts_data, 10, -10, 0.3, "ts_ds.txt")
sampled_ts = random_downsample(ts_data, 19, -10, 0.17)

## prepare data for making kmz

**timeseries data format:**
```plaintext
-1   -1   -1   -1   date1  date2  date3 ...
num1 lon1 lat1 vel1 disp11 disp12 disp13 ...
num2 lon2 lat2 vel2 disp21 disp22 disp23 ...
...
```

In [ ]:
def prep_data_for_kmz(ts_data, date, out_vel_file, out_ts_file):
    first_line = np.asarray([[-1, -1, -1, -1]])
    first_line = np.hstack((first_line, date.reshape((1, -1))))
    out_ts = np.vstack((first_line, ts_data))
    out_vel = ts_data[0:, 0:4]

    print('writing data to {}'.format(out_ts_file))
    np.savetxt(out_ts_file, out_ts, fmt='%4f')
    print('done.')
    print('writing data to {}'.format(out_vel_file))
    np.savetxt(out_vel_file, out_vel, fmt='%4f')
    print('done.')

In [ ]:
prep_data_for_kmz(ts_data, date, 'vel.txt', 'ts.txt')
# prep_data_for_kmz(sampled_ts, date, 'vel_ds.txt', 'ts_ds.txt')

## make kmz

In [ ]:
!python3 make_kmz_timeseries.py -t ts_ds.txt -o ts_ds10.kmz

In [ ]:
!python3 make_kmz.py -v vel_ds.txt -o vel_ds.kmz

## plot timeseries displacement

**get displacement by number, must use complete ts_data (not downsampled)**

In [ ]:
def date2str(date):
    date_str = []
    for i in date:
        date_str.append(str(i))
    return date_str


def plot_displacement(num_list, ts_data, date, aspect=0.2, figsize=(15, 7), y_lim=[-100, 100], fig_name=None):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title('time series displacement', fontsize=40)
    ax.set_xlabel('date', fontsize=30)
    ax.set_ylabel('displacrment (mm)', fontsize=30)

    ax.set_ylim(y_lim[0], y_lim[1])
    ax.set_aspect(aspect)
    ax.minorticks_on()
    ax.xaxis.grid(True, which='both')
    ax.xaxis.set_tick_params(rotation=30, labelsize=15)
    ax.yaxis.grid(True, which='major')
    ax.yaxis.set_tick_params(rotation=0, labelsize=15)
    ax.set_xmargin(0.02)

    date = date2str(date)

    for num in num_list:
        disp = ts_data[num, 4:]
        ax.plot(date, disp, label=str(num), marker='o')
        ax.xaxis.set_ticks(date[::4])
    ax.legend(loc='best', fontsize=20, ncol=2)
    fig.show()
    if fig_name:
        fig.savefig(fig_name, dpi=200)

In [ ]:
num_list = [113174]
plot_displacement(num_list, ts_data, date, aspect=0.2,
                  figsize=(30, 15), y_lim=[-150, 10], fig_name=None)

## cut velocity and timeseries

**find points inside the polygon**

In [ ]:
def intersect(point, s_point, e_point):
    if s_point[1] == e_point[1]:  # parallel and coincident with the ray，s_point coincides with s_point
        return False
    if s_point[1] > point[1] and e_point[1] > point[1]:  # line segment is above the ray
        return False
    if s_point[1] < point[1] and e_point[1] < point[1]:  # line segment under the ray
        return False
    if s_point[1] == point[1] and e_point[1] > point[1]:  # point coincides with s_point
        return False
    if e_point[1] == point[1] and s_point[1] > point[1]:  # point coincides with e_point
        return False
    # line segment is to the left of the ray
    if s_point[0] < point[0] and e_point[1] < point[1]:
        return False

    xseg = e_point[0]-(e_point[0]-s_point[0])*(e_point[1]-point[1]) / \
        (e_point[1]-s_point[1])  # find the intersection
    if xseg < point[0]:  # intersection is to the left of point
        return False
    return True


def inpolygon(point, polygon):
    num = 0  # number of intersection
    for i in range(len(polygon)-1):
        if intersect(point, polygon[i], polygon[i+1]):
            num += 1
    return True if num % 2 == 1 else False

### kml2polygon

In [ ]:
def kml2polygon(kml_file, polygon_file):
    cmd_str = f"gmt kml2gmt {kml_file} | awk 'NR>1' > {polygon_file}"
    os.system(cmd_str)
    print("done.")

In [ ]:
os.chdir('/media/ly/file/读取MintPy结果/cut')
kml2polygon('cut1.kml', 'cut1.txt')

### cut velocity

In [ ]:
def cut_vel(polygon_file, vel_file, out_vel_file):
    polygon = np.loadtxt(polygon_file)
    vel = np.loadtxt(vel_file)
    out_data = np.arange(vel.shape[1])
    for line in vel:
        if inpolygon(line[1:3], polygon):
            out_data = np.vstack((out_data, line))
    np.savetxt(out_vel_file, out_data[1:, :], fmt='%4f')
    print('done.')

In [ ]:
cut_vel('cut1.txt', 'vel.txt', 'vel_cut1.txt')

In [ ]:
!python3 make_kmz2.py -v vel_cut1.txt -o vel_cut1.kmz

### cut timeseries

In [ ]:
def cut_ts(polygon_file, ts_file, out_ts_file):
    polygon = np.loadtxt(polygon_file)
    data = np.loadtxt(ts_file)
    ts = data[1:, :]
    out_data = data[0, :]
    for line in ts:
        if inpolygon(line[1:3], polygon):
            out_data = np.vstack((out_data, line))
    np.savetxt(out_ts_file, out_data, fmt='%4f')
    print('done.')

In [ ]:
os.chdir('/media/ly/file/读取MintPy结果/cut')
cut_ts('cut1.txt', 'ts.txt', 'ts_cut1.txt')

In [ ]:
!python3 make_kmz_timeseries2.py -t ts_cut1.txt -o ts_cut1.kmz -s 0.8